In [1]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models
from glob import glob
from torch.utils.data import DataLoader, Dataset
import numpy as np
from torchsummary import summary
import torch.nn.functional as F
import torch.utils.data as utils
from tqdm import tqdm
from os.path import isfile, join
from os import listdir
import datetime
import pandas as pd
import time, copy
from models_pytorch import CNNT5, CNNT4
from vgg_pytorch import vgg11
from data_preprocessing.dataloaders_pytorch import LUNA_Dataset
from train_tools import train_model

LOGS = '/home.stud/dobkomar/NoduleDetection/logs/'


def get_free_gpu():
    os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
    memory_available = [int(x.split()[2]) for x in open('tmp', 'r').readlines()]
    print(memory_available)
    os.remove('tmp')
    return np.argmax(memory_available)

/mnt/home.stud/dobkomar/nodule_detection/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'mtrand' does not match runtime version 3.5
  return f(*args, **kwds)


### training parameters configurations

In [8]:
num_epochs = 30
num_classes = 2
batch_size = 64
learning_rate = 0.0005
shuffle_dataset = True

### load data

In [14]:
train_data = pd.read_csv('/home.stud/dobkomar/data/train_D1.csv')
val_data = pd.read_csv('/home.stud/dobkomar/data/test_D1.csv')


# IF 2-D DATASET
train_dataset = LUNA_Dataset(train_data)
val_dataset = LUNA_Dataset(val_data)

# ELIF 3-D DATASET
# train_dataset = LUNA_Dataset_3D(train_data)
# val_dataset = LUNA_Dataset_3D(val_data)


train_loader = utils.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = utils.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# train_iter, val_iter = iter(train_loader), iter(validation_loader)
# images_tr, label_tr = train_iter.next()
# images_v, labels_v = val_iter.next()

# print('TRAIN Images shape on batch size = {}'.format(images_tr.size()))
# print('TRAIN Labels shape on batch size = {}'.format(label_tr.size()))
# print('TRAIN shape: {}'.format(len(train_loader)))
# print('VAL Images shape on batch size = {}'.format(images_v.size()))
# print('VAL Labels shape on batch size = {}'.format(labels_v.size()))
# print('VAL shape: {}'.format(len(validation_loader)))

### set GPU 

In [3]:
device = torch.device("cuda:{}".format(get_free_gpu()))

[11178, 11178, 11178, 11178, 395, 395, 395, 395]


# All the models except for VGG

In [4]:
model = CNNT4().to(device)  # HERE YOU CAN PUT ANY NETWORK FROM module models_pytorch.py
print(model)

CNNT4(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=400, out_features=150, bias=True)
  (fc2): Linear(in_features=150, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=50, bias=True)
  (fc_out): Linear(in_features=50, out_features=2, bias=True)
)


In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
# learning_rate = 0.0005
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.95)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0005, momentum=0.95)

criterion = nn.CrossEntropyLoss()   # weight=class_weights)

dataloaders_dict = {'train': train_loader, 'val': validation_loader}

# Train and evaluate
model, hist, df_logs = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

# Saving results and model
df_logs.to_csv('/home.stud/dobkomar/NoduleDetection/logs/models/convnet4_D1.csv', index=False)
name = 'convnet4_D1'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, '/home.stud/dobkomar/NoduleDetection/logs/models/' + '{}.pt'.format(name))
print('model saved')

# VGG

In [20]:
# INITIALIZE MODEL 

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


# VGG11
feature_extract = False
model = vgg11(pretrained=False)
set_parameter_requires_grad(model, feature_extract)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)
input_size = 32

num_features = model.classifier[6].in_features
features = list(model.classifier.children())[:-1]   # Remove last layer
model = model.to(device)
features[0] = nn.Linear(2048, 4096)
features.extend([nn.Linear(4096, 2)])    # Add our layer with 4 outputs
model.classifier = nn.Sequential(*features)   # Replace the model classifier
model.classifier.requires_grad = True

In [21]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 512, kern

In [22]:
# TRAIN MODEL 

# Send the model to GPU
model = model.to(device)
params_to_update = model.parameters()


# Observe that all parameters are being optimized
# optimizer = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(params_to_update, lr=0.0005)
criterion = nn.CrossEntropyLoss()
dataloaders_dict = {'train': train_loader, 'val': validation_loader}

# Train and evaluate
model, hist, df_logs = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

In [23]:
# Saving results and model
df_logs.to_csv('/home.stud/dobkomar/NoduleDetection/logs/models/vgg11_D1.csv', index=False)
name = 'vgg11_D1'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, '/home.stud/dobkomar/NoduleDetection/logs/' + '{}.pt'.format(name))
print('model saved')